
This notebook demonstrates the minimal workflow:

1. Create a KnowledgeGraph
2. Explore a concept neighbourhood
3. Find and rank paths
4. Explain why a path was selected

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv

from kg_core.graph.scoring import find_shortest_paths, rank_paths, explain_path
from kg_core.graph.traverse import traverse
from kg_core.graph.paths import find_shortest_paths
from kg_core.graph.kg import KnowledgeGraph
from kg_core.graph.edges import PredicateKind
from kg_core.render import (
    render_subgraph,
    render_trace,
    render_path,
    render_explained_path,
    bind_default_renderers,
)

from omop_alchemy import configure_logging, get_engine_name, TEST_PATH, ROOT_PATH
import sqlalchemy as sa


In [2]:
configure_logging()
load_dotenv()

True

In [3]:
engine_string = get_engine_name()
engine = sa.create_engine(engine_string, future=True, echo=False)

2026-01-05 14:37:28,284 | INFO     | omop_alchemy.omop_alchemy.config | Database engine configured


In [4]:
Session = sessionmaker(bind=engine)

session = Session()
kg = KnowledgeGraph(session)
bind_default_renderers(kg)

In [5]:
ingredient = kg.concept_id_by_code("RxNorm", "6809") # Metformin
drug = kg.concept_id_by_code("RxNorm", "860975") # Metformin 500 MG Oral Tablet

In [6]:
kg.concept_view(drug)

ConceptView(id=40163924, RxNorm:860975, name='24 HR metformin hydrochloride 500 MG Extended Release Oral Tablet')

In [7]:
kg.concept_view(ingredient)

ConceptView(id=1503297, RxNorm:6809, name='metformin')

## Note on domain scope:

By default, traversal is restricted to relationships within the same OMOP domain (e.g. Drug -> Drug, Condition -> Condition). This avoids misleading “shortcut” paths through terminology metadata. 

Cross-domain reasoning (e.g. Drug -> Condition) requires explicit biomedical relationships and is intentionally out of scope for the default - it also frequently defaults to structural or metadata relationsips such as SNOMED 'has module' which becomes rapidly non-specific if all terms resolve in 1 or 2 steps to an extreme high level parent.


In [8]:
paths, trace = find_shortest_paths(
    kg,
    source=drug,
    target=ingredient,
    predicate_kinds={
        PredicateKind.ONTOLOGICAL,
        PredicateKind.MAPPING,
    },
    max_depth=6,
    traced=True,
)

In [9]:
ranked = rank_paths(kg, paths)

In [10]:
from IPython.display import HTML, display

display(HTML(render_trace(kg, trace)))


In [11]:
len(ranked)

2

In [12]:
ranked[0]

GraphPath(steps=(PathStep(subject=40163924, predicate='RxNorm is a', object=36223144), PathStep(subject=36223144, predicate='RxNorm has ing', object=1503297)))

In [17]:
explain_path(kg, ranked[0], trace)

PathExplanation(path=GraphPath(steps=(PathStep(subject=40163924, predicate='RxNorm is a', object=36223144), PathStep(subject=36223144, predicate='RxNorm has ing', object=1503297))), profile=PathProfile(hops=2, invalid_concepts=0, non_standard_concepts=0, vocab_switches=0, ontological_edges=2, mapping_edges=0, metadata_edges=0), steps=(PathExplanationStep(step=PathStep(subject=40163924, predicate='RxNorm is a', object=36223144), traversal_depth=0, predicate_kind=<PredicateKind.ONTOLOGICAL: 1>, reason='ontological relationship (preferred structure)'), PathExplanationStep(step=PathStep(subject=36223144, predicate='RxNorm has ing', object=1503297), traversal_depth=None, predicate_kind=<PredicateKind.ONTOLOGICAL: 1>, reason='ontological relationship (preferred structure)')))

In [15]:
for p in ranked:
    display(HTML(render_path(kg, p)))